In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-129497")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-129497
Azure region: southcentralus
Subscription id: 8a56ebc5-caa7-4a43-b8ed-1c1895e46051
Resource group: aml-quickstarts-129497


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Reference: lesson 5.9 and Microsoft website at:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import normal,uniform,choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( 
     {
    '--C': choice(0.0001,0.001,0.01,0.1,1,10,100,1000),
    '--max_iter': choice(100,200,300,400,500)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".",compute_target=cpu_cluster,entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [31]:
import joblib


# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()

best_run_metrics = best_run.get_metrics()
params = best_run.get_details()['runDefinition']['arguments']
print('Id of the best run: ', best_run.id)
print('Accuracy of the best run:', best_run_metrics['Accuracy'])


best_run.get_file_names()

Id of the best run:  HD_75bc379f-ff34-4be1-9be5-c41d7fedf573_11
Accuracy of the best run: 0.9036418816388467


['azureml-logs/55_azureml-execution-tvmps_47bb75b5ae5e3a9687fcb36875e81b41abb18fdd6fbba80ba7b039d917d5374a_d.txt',
 'azureml-logs/65_job_prep-tvmps_47bb75b5ae5e3a9687fcb36875e81b41abb18fdd6fbba80ba7b039d917d5374a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_47bb75b5ae5e3a9687fcb36875e81b41abb18fdd6fbba80ba7b039d917d5374a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/104_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/bestrun.joblib']

In [32]:
import joblib
# insert this after fitting the model
# create an output folder
model = best_run.register_model(model_name='hd_model', model_path='azureml-logs/model.pkl')


In [33]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

datastore_path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)




In [34]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

from sklearn.model_selection import train_test_split

import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y)

training_data = pd.concat([x_train, y_train], axis=1)

In [35]:
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
18008,23,0,0,1,0,5,5,16,5,999,...,0,0,0,0,1,0,0,0,0,0
24817,30,1,0,0,0,8,3,345,2,999,...,0,0,0,0,0,0,0,1,0,0
968,32,1,0,0,0,4,4,130,1,999,...,1,0,0,0,0,0,0,1,0,0
13856,24,0,0,0,1,5,2,140,2,999,...,1,0,0,0,0,0,1,0,0,0
15591,31,1,0,1,0,5,3,287,1,999,...,1,1,0,0,0,0,0,0,0,0


In [36]:
training_data.describe()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
count,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,...,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000,24712.000000
mean,40.056774,0.603674,0.000081,0.521528,0.151343,6.610756,2.981628,256.868283,2.559162,961.911905,...,0.364115,0.100558,0.054427,0.147256,0.230091,0.000324,0.128844,0.296455,0.042044,0.113386
std,10.482541,0.489143,0.008996,0.499546,0.358391,2.042573,1.411774,256.140750,2.767446,188.292945,...,0.481191,0.300749,0.226863,0.354368,0.420899,0.017990,0.335034,0.456703,0.200694,0.317071
min,17.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2.000000,102.000000,1.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,38.000000,1.000000,0.000000,1.000000,0.000000,6.000000,3.000000,178.000000,2.000000,999.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,47.000000,1.000000,0.000000,1.000000,0.000000,8.000000,4.000000,318.000000,3.000000,999.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,98.000000,1.000000,1.000000,1.000000,1.000000,12.000000,5.000000,4918.000000,56.000000,999.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
training_data[training_data.isnull().any(axis=1)]

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y


In [38]:
training_data["y"].value_counts()

0    21910
1     2802
Name: y, dtype: int64

In [39]:
from azureml.core import Workspace, Dataset

#creating a folder
#dirname = "./training_data"
os.makedirs('training_data', exist_ok=True)

local_path = './training_data/training_data.csv'
training_data.to_csv(local_path)

# upload the local file to a datastore on the cloud
workspace = Workspace(ws.subscription_id, ws.resource_group, ws.name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training_data', target_path='training_data')

# create a dataset referencing the cloud location

ds = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('training_data/training_data.csv'))])

Uploading an estimated of 1 files
Uploading training_data/training_data.csv
Uploaded training_data/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [41]:
# Submit your automl run
exp = Experiment(workspace=ws, name="AutoML-project")
automl_run=exp.submit(automl_config,show_output=True)
RunDetails(automl_run).show()
assert(automl_run.get_status() == "Completed")

Running on local machine
Parent Run ID: AutoML_c4540965-3ad2-45c6-ae16-0f0acfcf4c49

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [46]:
# Retrieve and save your best automl model.


automl_best_run, fitted_automl_model = automl_run.get_output() 
print("Fitted model: ", fitted_automl_model.steps[-1])
print("---"*5)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("---"*5)
print("AutoML run Summary: ", automl_run.summary())

Fitted model:  ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                  

In [47]:
cpu_cluster.delete()